##### 多分类问题
- softmax分类器
[![xgH0EQ.jpg](https://s1.ax1x.com/2022/10/23/xgH0EQ.jpg)](https://imgse.com/i/xgH0EQ)
- 分布：输出之间具有竞争性，归一化


In [3]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

In [4]:
batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), # 通道转换
                                transforms.Normalize((0.1307,),(0.3081,))]) #均值，归一化，映射到 N（0，1）
train_dataset = datasets.MNIST(root='../dataset/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(root='../dataset/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)


In [5]:
#模型
class Net(torch.nn.Module):
    def __init__(self):                                                                                                                                        
        super(Net, self).__init__()                                                                    
        self.l1 = torch.nn.Linear(784, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)                                                             
        self.l5 = torch.nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # -1其实就是自动获取mini_batch
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)  # 最后一层不做激活，不进行非线性变换

model = Net()
#损失，优化器
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.5)

In [6]:
#训练
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        # 获得一个批次的数据和标签
        inputs, target = data
        optimizer.zero_grad()
        # 获得模型预测结果(64, 10)
        outputs = model(inputs)
        # 交叉熵代价函数outputs(64,10),target（64）
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0

In [7]:
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1) # dim = 1 列是第0个维度，行是第1个维度
            total += labels.size(0)
            correct += (predicted == labels).sum().item() # 张量之间的比较运算
    print('accuracy on test set: %d %% ' % (100*correct/total))

if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 2.146
[1,   600] loss: 0.752
[1,   900] loss: 0.408
accuracy on test set: 90 % 
[2,   300] loss: 0.297
[2,   600] loss: 0.258
[2,   900] loss: 0.221
accuracy on test set: 94 % 
[3,   300] loss: 0.182
[3,   600] loss: 0.166
[3,   900] loss: 0.152
accuracy on test set: 95 % 
[4,   300] loss: 0.126
[4,   600] loss: 0.126
[4,   900] loss: 0.119
accuracy on test set: 96 % 
[5,   300] loss: 0.095
[5,   600] loss: 0.095
[5,   900] loss: 0.102
accuracy on test set: 96 % 
[6,   300] loss: 0.074
[6,   600] loss: 0.080
[6,   900] loss: 0.077
accuracy on test set: 97 % 
[7,   300] loss: 0.062
[7,   600] loss: 0.064
[7,   900] loss: 0.063
accuracy on test set: 97 % 
[8,   300] loss: 0.050
[8,   600] loss: 0.053
[8,   900] loss: 0.047
accuracy on test set: 97 % 
[9,   300] loss: 0.041
[9,   600] loss: 0.043
[9,   900] loss: 0.042
accuracy on test set: 97 % 
[10,   300] loss: 0.030
[10,   600] loss: 0.030
[10,   900] loss: 0.038
accuracy on test set: 97 % 


#####  人工特征提取:傅里叶，小波变换
##### 自动特征提取： CNN